In [1]:
%config Completer.use_jedi = False

In [2]:
#import tensorflow as tf

import numpy as np
import pandas as pd

import glob, os

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
%%time
file_name = '../FullCohort_v3_07MAR2016.csv'
df = pd.read_csv(
    file_name,
    encoding='ISO-8859-1',
    low_memory=False,
)

CPU times: user 3min 3s, sys: 13.1 s, total: 3min 16s
Wall time: 3min 43s


In [4]:
df.shape

(3400056, 462)

In [5]:
numeric_IDs = [
    'HospitalID','VisitKey', 'EpisodeKey', 'ZipCode', 'SubmissionPatientKey', 'ProcedureDate', 'NCDRPatientID'
]

numeric_remove = [
    'SubmissionKey', #'SubmissionPatientKey',
    'DeathCause', 'DeathLab','ZipCodeNA', 'HospStatus', 'ArrivalDate',
    'PriorPCIDate', 'PriorCABGDate', 'CABGDate', 'DCDate', 'ArrivalTime', 'CABGTime',
    'DOB', 'TimeframeCode', 'PopulationCode', 'BenchmarkCode', 
    'STATE','MPN', 'ClientName.x', 'H_NPI', 'H_AHA', 'Address1.x',
    'Address2.x', 'City.x', 'Zip.x', 'CPMPN', 'ICDMPN', 'ACTMPN', 'mpn2', 
    'PROV_NUM', 'MPNDUP', 'PCI', 'OnsetDate', 'ThromDate', #'ProcedureDate', 
    'SubECGDate', 'FirstDevActiDate', 'EDPresentDate', 'OnsetTime', 'ThromTime',
    'ProcedureTime', 'SubECGTime','FirstDevActiTime',
    'EDPresentTime', 'PrevLesionTime','HID', 'PID', 'EID', 'ClientName.y', 
    'ParticipantClassificationDesc', 'ParticipantClassOtherDesc', 'Address1.y', 
    'Address2.y', 'City.y', 'Zip.y', 'CountryCode', 'CountryName', 'ISOCode2', 
    'ISONumber','StateProvinceCode', 'StateProvinceName', 'MasterContractDate', 
    'MasterContractEffectiveDate', 'ContractDate', 'ContractEffectiveDate',
    'PaidThroughDate', 'DateLastModified', 'iMISID', 'Hosp_ClientName', 'Hosp_ H_AHA',
    'Hosp_ H_NPI', 'Hosp_ CITY', 'Hosp_ STATE', 'Hosp_ MPN'
]

numeric_exc = [
    'exc1', 'exc2','exc3', 'exc4', 'exc5', 'exc6'
]

numeric_other = ['DCathOperatorKey' ,'PCIOperatorKey']

numeric_insurance = [
    'InsPrivate', 'InsMedicare' ,'InsMedicaid', 'InsMilitary','InsState', 
    'InsIHS', 'InsNonUS', 'InsNone'
]

In [6]:
numeric_blanks = ['CABGLocation', 'CABGIndication', 'CABGStatus']

In [7]:
med_vars = [col for col in df.columns if 'PreOp' in col]

In [8]:
dev_na_cols = med_vars + [
    "ICDEV_Drug Eluting Stent" , "ICDEV_Balloon" , "ICDEV_Bare Metal Stent" , "ICDEV_Cutting Balloon" ,
    "ICDEV_Thrombectomy" , "ICDEV_Extraction Catheter" , "ICDEV_Embolic Protection" , "ICDEV_Atherectomy" ,
    "ICDEV_Laser" , "ICDEV_Other" , "ICDEV_Coated Stent" , "ICDEV_Chronic Total Occlusion", "ICDEV_Brachy Therapy" ,
    "ICDEV_Covered Stent" , "LesionCounter.y" , "LSDEV_Drug Eluting Stent" , "LSDEV_Balloon" , "LSDEV_Bare Metal Stent" ,
    "LSDEV_Cutting Balloon" , "LSDEV_Thrombectomy" , "LSDEV_Extraction Catheter" , "LSDEV_Embolic Protection" ,
    "LSDEV_Atherectomy" , "LSDEV_Laser" , "LSDEV_Other" , "LSDEV_Coated Stent" , "LSDEV_Chronic Total Occlusion",
    "LSDEV_Brachy Therapy" , "LSDEV_Covered Stent" , "ClosID1" , "ClosID10" , "ClosID11" , "ClosID12" , "ClosID13" ,
    "ClosID14" , "ClosID15" , "ClosID16" , "ClosID17", "ClosID18" , "ClosID19" , "ClosID2" , "ClosID20" , "ClosID21" ,
    "ClosID22" , "ClosID23" , "ClosID24" , "ClosID25" , "ClosID26" , "ClosID27" , "ClosID28" , "ClosID29" , "ClosID3" ,
    "ClosID30", "ClosID31" , "ClosID32" , "ClosID33" , "ClosID34" , "ClosID35", "ClosID36" , "ClosID37" , "ClosID38" ,
    "ClosID39" , "ClosID4" , "ClosID40" , "ClosID41" , "ClosID42" , "ClosID43" , "ClosID44", "ClosID45" , "ClosID46" ,
    "ClosID47" , "ClosID48" , "ClosID49", "ClosID5" , "ClosID50" , "ClosID51" , "ClosID52" , "ClosID53", "ClosID54" ,
    "ClosID55" , "ClosID56" , "ClosID57" , "ClosID58", "ClosID59" , "ClosID6" , "ClosID60" , "ClosID61" , "ClosID62",
    "ClosID63" ,"ClosID64" , "ClosID65" , "ClosID66" , "ClosID68", "ClosID69" , "ClosID7" , "ClosID70" , "ClosID71" ,
    "ClosID72", "ClosID8" , "ClosID9" , "ClosID952" , "ClosID953" , "ClosID954", "ClosID955"
]

In [9]:
categorical_drops = ['NPINumber' ,'AHANumber', 'CommunityDesc', 'ProfitTypeDesc', 
                     'HemoDynamicEquipment1','HemoDynamicEquipment2',
                     'HemoDynamicEquipment3', 'HemoDynamicEquipment4']

In [10]:
dq_cols = [col for col in df.columns if 'DQ' in col]

In [11]:
df['STEMI'] = (df.CADPresentation == 6).astype(int)

In [12]:
fillna_0_cols = numeric_insurance + numeric_blanks + dev_na_cols

In [13]:
df[fillna_0_cols] = df[fillna_0_cols].fillna(0)

In [14]:
df[med_vars] = df[med_vars].replace({3:0, 2:0, 5:0})

In [15]:
df_excludes = df[numeric_exc].copy()

In [16]:
df = df.drop(
    columns= numeric_remove + numeric_exc + numeric_other + categorical_drops + dq_cols
)

In [17]:
dummy_encode = [
    'SegmentID',
    'AdmtSource',
    'Dominance',
    'LocGraft',
    'PCIDelayReason',
    'PCIndication',
]
df['AdmtSource'] = df['AdmtSource'].fillna(3)

In [18]:
for col in dummy_encode:
    try:
        df[col] = df[col].astype('Int32')
    except:
        print(col)

SegmentID


In [19]:
df = pd.get_dummies(df, columns=dummy_encode)

In [20]:
df.head()

,HospitalID,VisitKey,EpisodeKey,SubmissionPatientKey,ZipCode,CABGLocation,CABGIndication,CABGStatus,DCStatus,Smoker,...,PCIDelayReason_5,PCIDelayReason_6,PCIndication_1,PCIndication_2,PCIndication_3,PCIndication_4,PCIndication_5,PCIndication_6,PCIndication_7,PCIndication_8
0,100270,43745369.0,108993902.0,34101796.0,30601,0.0,0.0,0.0,1.0,1.0,...,0,1,1,0,0,0,0,0,0,0
1,100270,45509181.0,115038766.0,35374034.0,30075,1.0,4.0,2.0,1.0,0.0,...,0,1,1,0,0,0,0,0,0,0
2,100270,36250681.0,82732100.0,28577512.0,30677,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,1
3,100270,45509190.0,115038775.0,35375266.0,78577,0.0,0.0,0.0,1.0,0.0,...,0,1,1,0,0,0,0,0,0,0
4,100270,4366478.0,4242004.0,4131444.0,30606,0.0,0.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,1


In [21]:
df.loc[df.CTO == -np.inf, 'CTO'] = np.nan

df.loc[df.PreTIMI == np.inf, 'PreTIMI'] = np.nan

df.loc[df.NEWSEQ == -np.inf, 'NEWSEQ'] = np.nan

In [22]:
df['DiabetesControl'].value_counts(dropna=False)

NaN    2144319
3.0     629824
4.0     476242
2.0      79363
1.0      65344
5.0       4964
Name: DiabetesControl, dtype: int64

In [23]:
df.loc[(df.Diabetes == 0) & (df.DiabetesControl.isna()), 'DiabetesControl'] = 1

In [24]:
df['DiabetesControl'] = df['DiabetesControl'].fillna(5)

In [25]:
df['DiabetesControl'].value_counts(dropna=False)

1.0    2206273
3.0     629824
4.0     476242
2.0      79363
5.0       8354
Name: DiabetesControl, dtype: int64

In [26]:
df['DiabetesControl'].replace({
    1:'None',
    2:'Diet',
    3:'Oral',
    4:'Insulin',
    5:'OtherOrMissing'
}, inplace=True)

In [27]:
df['DiabetesControl'].value_counts(dropna=False)

None              2206273
Oral               629824
Insulin            476242
Diet                79363
OtherOrMissing       8354
Name: DiabetesControl, dtype: int64

In [28]:
df = pd.get_dummies(df, columns=['DiabetesControl'])

Create Rao Variables

In [29]:
df['AGEGT70'] = (df.Age > 70).astype(int)
df['AGELE70'] = (df.Age <= 70).astype(int)

df['BMI'] = df.Weight * 10000 / (df.Height * df.Height) #Factor of 10000 for conversion from cm^2 to m^2
df.loc[(df.BMI < 5) | (df.BMI > 100), 'BMI'] = np.nan

df['BMILE30'] = (df.BMI <= 30).astype(int)

dm_vars = [
    'DiabetesControl_Insulin',
    'DiabetesControl_Diet',
    'DiabetesControl_Oral',
    'DiabetesControl_OtherOrMissing'
]

df['NEWDIAB'] = (df.Diabetes == 1).astype(int) * (df[dm_vars].any(axis=1).astype(int) + df[dm_vars[0]])

#translated from
#NEWDIAB <- rep(0, dim(ncdr.data.full)[1])
#NEWDIAB[which((final.complete.input$Diabetes == 1) & (final.complete.input$DiabetesControl > 1))] <- 1
#NEWDIAB[which((final.complete.input$Diabetes == 1) & (final.complete.input$DiabetesControl == 4))] <- 2
#
#ncdr.data.full$NEWDIAB <- NEWDIAB
#
#NEWDIAB1 <- as.numeric(NEWDIAB == 1)
#NEWDIAB2 <- as.numeric(NEWDIAB == 2)
#
#ncdr.data.full$NEWDIAB1 <- NEWDIAB1
#ncdr.data.full$NEWDIAB2 <- NEWDIAB2

df['NEWDIAB1'] = (df['NEWDIAB'] == 1).astype(int)
df['NEWDIAB2'] = (df['NEWDIAB'] == 2).astype(int)

In [30]:
df['HDEF'] = df.PrePCILVEF

df.loc[df['HDEF'] > 60, 'HDEF'] = 60

df['HDEF'] = df['HDEF'] / 5

In [31]:
df['FEMALE'] = (df['Sex']==2).astype(int)

df['GENDMULT'] = 1 - 0.258*df['FEMALE']

df['RACEMULT'] = 1 + 0.21 * df['RaceBlack']

#GFR[idx] <- (186 * (ncdr.data.full$PreProcCreat[idx] ^ (-1.154))) * (ncdr.data.full$Age[idx] ^ (-0.203)) * GENDMULT[idx] * RACEMULT[idx]
#
#RENFAIL <- as.numeric((GFR < 30) | (ncdr.data.full$CurrentDialysis == 1))
#
#ncdr.data.full$RENFAIL <- RENFAIL


GFR = (186 * (df.PreProcCreat.pow(-1.154))) * (df.Age.pow(-0.203)) * df.GENDMULT * df.RACEMULT

df['RENFAIL'] = ((GFR < 30) | (df.CurrentDialysis == 1)).astype(int)

In [32]:
df['CKD'] = 3
df.loc[(GFR >= 30), 'CKD'] = 2
df.loc[(GFR >= 45), 'CKD'] = 1
df.loc[(GFR >= 60), 'CKD'] = 0
df.loc[(df.CurrentDialysis == 1) | (GFR < 30), 'CKD'] = 4

In [33]:
df.CKD.value_counts()

0    2357236
1     498918
2     211942
3     189103
4     142857
Name: CKD, dtype: int64

In [34]:
df['CKD1'] = (df.CKD == 1).astype(int)
df['CKD2'] = (df.CKD == 2).astype(int)
df['CKD3'] = (df.CKD == 3).astype(int)
df['CKD4'] = (df.CKD == 4).astype(int)

In [35]:
df['GFR'] = np.minimum(GFR, 90)/5

In [36]:
df['LYTICS'] = ((df.CADPresentation == 6) & (df.ThromTherapy == 1)).astype(int)

In [37]:
df['CARSHOCK'] = ((df.PriorCardioShock == 1) & (df.PCICardioShock == 1)).astype(int)

In [38]:
df['DCARSHOCK'] = 0
df.loc[df.PriorCardioShock == 1, 'DCARSHOCK'] = 1 + (df.PCICardioShock == 0).astype(int)
df.loc[df.PriorCardioShock == 0, 'DCARSHOCK'] = 3 + (df.PCICardioShock == 0).astype(int)

In [39]:
SHOCKPCIS = pd.Series(6, index=df.index)
SHOCKPCIS.loc[df.PCIStatus == 2] = 5
SHOCKPCIS.loc[df.PCIStatus == 3] = 4
SHOCKPCIS.loc[(df.DCARSHOCK == 2)|(df.DCARSHOCK == 3)] = 3
SHOCKPCIS.loc[(df.DCARSHOCK == 1)|(df.PCIStatus == 4)] = 2
SHOCKPCIS.loc[(df.DCARSHOCK == 1)&(df.PCIStatus == 4)] = 1

In [40]:
df['SHOCKPCIS'] = SHOCKPCIS.copy()
df['SHOCKPCIS1'] = (SHOCKPCIS == 1).astype(int)
df['SHOCKPCIS2'] = (SHOCKPCIS == 2).astype(int)
df['SHOCKPCIS3'] = (SHOCKPCIS == 3).astype(int)
df['SHOCKPCIS4'] = (SHOCKPCIS == 4).astype(int)
df['SHOCKPCIS5'] = (SHOCKPCIS == 5).astype(int)
df['SHOCKPCIS6'] = (SHOCKPCIS == 6).astype(int)

In [41]:
df['LESSCAI23'] = (df.LESSCAI > 1).astype(int)
#Note- original has LESSCAI4, but no pts in this data have == 4

In [42]:
df.LESSCAI.value_counts()

2    1384329
1    1358049
3     657678
Name: LESSCAI, dtype: int64

In [43]:
df['NEWSEQ2'] = (df.NEWSEQ == 2).astype(int)
df['NEWSEQ3'] = (df.NEWSEQ == 3).astype(int)

In [44]:
df['NYHA123'] = ((df.Prior2weekNYHA == 1) | (df.Prior2weekNYHA == 2) | (df.Prior2weekNYHA == 3)).astype(int)
df['NYHA4'] = (df.Prior2weekNYHA == 4).astype(int)

In [45]:
df['PRETIMINO'] = (df.PreTIMI  == 0).astype(int)

In [46]:
NVD = pd.Series(0, index=df.index)
NVD.loc[df.LMStenosis <= 50] = (
     ((df.RCAStenosis > 70).astype(int) + 
     ((df.ProxLADStenosis > 70) | (df.MidDistalLADStenosis > 70)).astype(int) +
     ((df.CIRCStenosis > 70) | (df.RamusStenosis > 70))).astype(int)
)[df.LMStenosis <= 50]
NVD.loc[(df.LMStenosis > 50) &  ((df.Dominance_1 == 1) | (df.RCAStenosis > 70))] = 3
NVD.loc[(df.LMStenosis > 50) & ~((df.Dominance_1 == 1) | (df.RCAStenosis > 70))] = 2

df['NVD'] = NVD.copy()
df['NVD23'] = (NVD > 1).astype(int)

In [47]:
df['PREHGBLE13'] = (df.PreProcHgb <= 13).astype(int)
df['PREHGBGT13'] = (df.PreProcHgb >  13).astype(int)

In [48]:
file_name = '../v4closm.sas7bdat.csv'
data_sas = pd.read_csv(
    file_name,
    encoding='ISO-8859-1',
    low_memory=False,
)

data_sas.VisitKey = data_sas.VisitKey.astype(int)

In [49]:
closure_dict = data_sas.drop_duplicates(subset='ClosureDevID')[['ClosureDevID', 'DeviceTypeName']].reset_index(drop=True)

closure_dict = closure_dict.set_index('ClosureDevID').to_dict()['DeviceTypeName']

In [50]:
closure_ids = [col for col in df.columns if 'ClosID' in col]

In [51]:
invert_dict = {}
for k in set(closure_dict.values()):
    invert_dict[k] = []
for k,v in closure_dict.items():
    invert_dict[v].append(f'ClosID{k}')

In [52]:
invert_dict.keys()

dict_keys(['Other', 'Manual com', 'Suture', 'Staple', 'Sealant', 'Patch', 'Mechanical'])

In [53]:
closure_df = pd.DataFrame(index=df.index, dtype=int)
for col in invert_dict.keys():
    closure_df['closure_' + col.replace(' ', '_')] = df[invert_dict[col]].astype(int).sum(axis=1)

In [54]:
for col in closure_df.columns:
    print(col)

closure_Other
closure_Manual_com
closure_Suture
closure_Staple
closure_Sealant
closure_Patch
closure_Mechanical


In [55]:
closure_df['total_count_closure'] = closure_df.sum(axis=1)
closure_df['closure_None'] = (closure_df['total_count_closure'] == 0).astype(int)

In [56]:
closure_df.total_count_closure.value_counts()

1     2932590
2      242746
0      199046
3       22539
4        2544
5         420
6         105
7          43
8          17
9           3
12          1
11          1
10          1
Name: total_count_closure, dtype: int64

In [57]:
df = pd.concat([df.drop(columns=closure_ids), closure_df], axis=1)

In [58]:
df.ProcedureDate = pd.to_timedelta(df.ProcedureDate, unit='D') + pd.Timestamp('1960-1-1')

In [59]:
(df.ProcedureDate < pd.Timestamp('2009-7-1')).sum()

3046

In [60]:
(df.ProcedureDate < pd.Timestamp('2009-1-8')).sum()

1

In [61]:
df_multiple_closures = df[
    (df.total_count_closure >= 2) |
    (df.ProcedureDate < pd.Timestamp('2009-7-1')) |
    (df.ClosureMethodNA==1)
].copy()
df_single_closure = df[
    (df.total_count_closure < 2) &
    (df.ProcedureDate >= pd.Timestamp('2009-7-1')) &
    (df.ClosureMethodNA != 1)
].copy()

In [62]:
df_multiple_closures.shape, df_single_closure.shape


((454598, 402), (2945458, 402))

Remove CABG Patients

In [70]:
cabg_cols = [col for col in df_single_closure.columns if 'CABG' in col and 'Prior' not in col]

df_single_closure = df_single_closure[df_single_closure.CABG!=1].drop(columns=cabg_cols)

Remove Other Major Surgery

In [71]:
df_single_closure = df_single_closure[df_single_closure.OtherMajorSurgery!=1].drop(columns='OtherMajorSurgery')

In [72]:
df_single_closure.total_count_closure.value_counts(dropna=False, sort=False)

0      14924
1    2864167
Name: total_count_closure, dtype: int64

In [73]:
df_single_closure.shape

(2879091, 397)

In [78]:
df_single_closure.AcessSite.value_counts(dropna=False)

1.0    2394173
3.0     474635
2.0       7950
4.0       1333
NaN       1000
Name: AcessSite, dtype: int64

In [79]:
df_final = df_single_closure[
    (df_single_closure.AcessSite == 1) |
    (df_single_closure.AcessSite == 3)
].copy()
df_final.shape

(2868808, 397)

In [81]:
df_final.NCDRPatientID.nunique()

1480091

In [83]:
df_final.HospitalID.nunique()

1538

In [87]:
df_final.AcessSite.value_counts(dropna=False)

1.0    2394173
3.0     474635
Name: AcessSite, dtype: int64

In [88]:
%%time
df_final['Femoral'] = (df_final.AcessSite == 1).astype(int)
df_final['Radial'] = (df_final.AcessSite == 3).astype(int)

df_final.drop(columns=['AcessSite', 'ClosureMethodNA', 'ClosureCounter'], inplace=True)

CPU times: user 1.69 s, sys: 936 ms, total: 2.62 s
Wall time: 2.62 s


In [89]:
[col for col in df_final.columns if 
 'clos' in col.lower() or
 'femor' in col.lower() or
 'radia' in col.lower()
]

['closure_Other',
 'closure_Manual_com',
 'closure_Suture',
 'closure_Staple',
 'closure_Sealant',
 'closure_Patch',
 'closure_Mechanical',
 'total_count_closure',
 'closure_None',
 'Femoral',
 'Radial']

In [90]:
for col in [col for col in df_final.columns if 
 'clos' in col.lower() or
 'femor' in col.lower() or
 'radia' in col.lower()
]:
    print(df_final[col].value_counts(dropna=False)/df_final.shape[0])
    print()

0    0.966949
1    0.033051
Name: closure_Other, dtype: float64

0    0.663408
1    0.336592
Name: closure_Manual_com, dtype: float64

0    0.907804
1    0.092196
Name: closure_Suture, dtype: float64

0    0.999936
1    0.000064
Name: closure_Staple, dtype: float64

0    0.680573
1    0.319427
Name: closure_Sealant, dtype: float64

0    0.96525
1    0.03475
Name: closure_Patch, dtype: float64

0    0.821191
1    0.178809
Name: closure_Mechanical, dtype: float64

1    0.994889
0    0.005111
Name: total_count_closure, dtype: float64

0    0.994889
1    0.005111
Name: closure_None, dtype: float64

1    0.834553
0    0.165447
Name: Femoral, dtype: float64

0    0.834553
1    0.165447
Name: Radial, dtype: float64



In [91]:
df_final.columns = [c.replace(' ', '_') for c in df_final.columns]

In [92]:
#2 = Unknown
df_final['CulpritArtery'] = df_final['CulpritArtery'].replace({2:np.nan})

In [93]:
#3 = Unknown
df_final['StentType'] = df_final['StentType'].replace({3:np.nan})

In [94]:
presentation_vars = list(pd.read_csv('presentation.txt', header=None)[0])
cath_lab_vars = list(pd.read_csv('cath_lab.txt', header=None)[0])
med_vars = list(pd.read_csv('meds.txt', header=None)[0])
pci_vars = list(pd.read_csv('pci.txt', header=None)[0])

In [95]:
df_final.Bleed.value_counts(dropna=False)

0    2750481
1     118327
Name: Bleed, dtype: int64

In [96]:
access_vars = [col for col in df_final.columns if 
               'femor' in col.lower() or
               'radia' in col.lower()]
closure_vars = [col for col in df_final.columns if 'clos' in col.lower()]

outcome_var = ['Bleed']

In [97]:
all_vars = outcome_var + presentation_vars + access_vars + cath_lab_vars + med_vars + pci_vars + closure_vars

In [98]:
len(all_vars) - len(set(all_vars))

0

In [99]:
for col in all_vars:
    if col not in df_final.columns:
        print(col)

In [101]:
df_train = df_final[df_final.ProcedureDate < pd.Timestamp('2014-1-1')]
df_test = df_final[df_final.ProcedureDate >= pd.Timestamp('2014-1-1')]

In [102]:
df_train.shape, df_test.shape

((2314446, 396), (554362, 396))

In [103]:
df_train.shape[0]/df_final.shape[0], df_test.shape[0]/df_final.shape[0]

(0.8067622510812853, 0.19323774891871467)

In [106]:
df_final.shape[0], df_final.NCDRPatientID.nunique(), df_final.HospitalID.nunique()

(2868808, 1480091, 1538)

In [104]:
df_train.shape[0], df_train.NCDRPatientID.nunique(), df_train.HospitalID.nunique()

(2314446, 1226682, 1486)

In [105]:
df_test.shape[0], df_test.NCDRPatientID.nunique(), df_test.HospitalID.nunique()

(554362, 436695, 1488)

In [108]:
df_reduced_test = df_test[~df_test.NCDRPatientID.isin(df_train.NCDRPatientID.unique())]

In [109]:
df_reduced_test.shape[0], df_reduced_test.NCDRPatientID.nunique(), df_reduced_test.HospitalID.nunique()

(281071, 253409, 1321)

In [111]:
df_test[['NCDRPatientID', 'HospitalID', 'ProcedureDate'] + all_vars].to_csv('data/pre_impute_test_jan_26_2021.csv')

In [112]:
df_train[['NCDRPatientID', 'HospitalID', 'ProcedureDate'] + all_vars].to_csv('data/pre_impute_train_jan_26_2021.csv')